In [1]:
import os

In [2]:
%pwd

'c:\\Work\\Tasks\\Repo\\uc_api\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Work\\Tasks\\Repo\\uc_api'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataValidationConfig:
    # Common Configurations
    root_dir: Path
    status_file: str
    
    # Common Parameters
    data_split_type: list 

In [6]:
from src.uc_api.constants import *
from src.uc_api.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
    ):
        # Reading yaml files for config and params
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_validation_config(self) -> DataValidationConfig:
        # Common configurations from yaml file
        config = self.config.data_validation
        
        # Common Parameters from yaml file
        params_common = self.params.CommonArguments

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            # Common Configurations
            root_dir=config.root_dir,
            status_file=config.status_file,
            
            # Common Parameters
            data_split_type=params_common.data_split_type,
        )

        return data_validation_config

In [8]:
import os
from src.uc_api.logging import logger

In [9]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config

    def validate_all_files_exist(self) -> bool:
        try:
            # Check for directories based on data split type example [train, test]
            validation_status = True
            path = os.path.join("artifacts", "data_ingestion", "processed")
            dataset_dirs = [f.name for f in os.scandir(path) if f.is_dir()]
            for dataset_type in dataset_dirs:
                if dataset_type not in self.config.data_split_type:
                    validation_status = False
            with open(self.config.status_file, "w") as f:        
                f.write(f"Validation status: {validation_status}")                        
                logger.info(f"Validation status: {validation_status}")
            return validation_status

        except Exception as e:
            raise e

In [10]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_files_exist()
except Exception as e:
    raise e

[2024-06-07 18:09:32: INFO:common: yaml file: config\config.yaml loaded successfully]
[2024-06-07 18:09:32: INFO:common: yaml file: params.yaml loaded successfully]
[2024-06-07 18:09:32: INFO:common: created directory at: artifacts]
[2024-06-07 18:09:32: INFO:common: created directory at: artifacts/data_validation]
[2024-06-07 18:09:32: INFO:2490338134: Validation status: True]
